In [15]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from cProfile import label #?not sure
import re
from fuzzywuzzy import fuzz

import difflib 
from fuzzywuzzy import process


### Feb 17 todo list:

- import basics from extract flora code
- organize fuzzy matching authors 
    - function for splitting authors into sets
    - functions for exact matching species name and fuzzy matching authors
    - functions for getting the match types 

### importing books and WCVP

In [2]:
# to fix unnecessary space at the end of authors (will fix index files later)
def fix_author(row):
    if not isinstance(row, str) or row[-1] != ' ':
        return row
    return row[:-1]

In [7]:
vol1_path = '../input/input copy/index_without_itallics_csvs/vol1/vol1_index_pruned.csv'
vol2_path = '../input/input copy/index_without_itallics_csvs/vol2/vol2_index_pruned.csv'
vol3_path = '../input/input copy/index_without_itallics_csvs/vol3/vol3_index_pruned.csv'

vol1_df = pd.read_csv(vol1_path)
vol2_df = pd.read_csv(vol2_path)
vol3_df = pd.read_csv(vol3_path)

#fix spacing issue
vol1_df['author'] = vol1_df['author'].apply(fix_author)
vol2_df['author'] = vol2_df['author'].apply(fix_author)
vol3_df['author'] = vol3_df['author'].apply(fix_author)

#changing name of columns 
vol1_df.rename(columns={'genus': 'mouterde_genus', 'epithet': 'mouterde_epithet', 'author':'mouterde_author', 'infra':'mouterde_infra'}, inplace=True)
vol2_df.rename(columns={'genus': 'mouterde_genus', 'epithet': 'mouterde_epithet', 'author':'mouterde_author', 'infra':'mouterde_infra'}, inplace=True)
vol3_df.rename(columns={'genus': 'mouterde_genus', 'epithet': 'mouterde_epithet', 'author':'mouterde_author', 'infra':'mouterde_infra'}, inplace=True)

#wcvp 
text_wcvp_path = '../input/input copy/wcvp/wcvp_v9_jun_2022.txt'
wcvp_df = pd.read_csv(text_wcvp_path, sep = '|')

### process wcvp (explode authors)

In [207]:
def get_set(x):
    if not isinstance(x, list): #is this the best way
        x = []
    return set(x)

#to use after exploding the authors
connectors = {'ex', '&', 'et'}
def remove_prantheses(x):
    if isinstance(x, str):
        return re.sub(r'([()])', '', x)
    return x

def process_wcvp(wcvp_df):
    wcvp_df['infraspecies'].replace(np.nan, 'np.NaN', inplace=True)
    wcvp_df['species'].replace(np.nan, 'np.NaN', inplace=True)
    wcvp_df['authors_set'] = wcvp_df['authors'].str.split(' ').apply(lambda x : get_set(x) - connectors)
    wcvp_df_explode = wcvp_df.explode('authors_set')
    wcvp_df_explode['authors_set_'] = wcvp_df_explode['authors_set'].apply(remove_prantheses) #to remove prantheses
    return wcvp_df_explode

wcvp_df_explode = process_wcvp(wcvp_df)

### process vols (explore authors)

just doing the set process during matching

In [23]:
"""def process_vols(df):
    delimeter = ' ' #TODO: FIX this absolutely HORRIBLE delimeter 
    df['authors_set'] = df['mouterde_author'].str.split(delimeter).apply(lambda x : get_set(x) - connectors)
    df_explode = df.explode('authors_set')
    df_explode['authors_set'] = df_explode['authors_set'].apply(remove_prantheses) #to remove prantheses
    return df_explode"""

In [26]:
"""vol1_explode = process_vols(vol1_df)
vol2_explode = process_vols(vol2_df)
vol3_explode = process_vols(vol3_df)"""

### Fixing Mout. authors

#### TODO: PUT THIS IN RIGHT ORDER reimports 

In [104]:
vol1_path = '../input/input copy/index_without_itallics_csvs/vol1/vol1_index_pruned.csv'
vol2_path = '../input/input copy/index_without_itallics_csvs/vol2/vol2_index_pruned.csv'
vol3_path = '../input/input copy/index_without_itallics_csvs/vol3/vol3_index_pruned.csv'

vol1_df = pd.read_csv(vol1_path)
vol2_df = pd.read_csv(vol2_path)
vol3_df = pd.read_csv(vol3_path)

#fix spacing issue
vol1_df['author'] = vol1_df['author'].apply(fix_author)
vol2_df['author'] = vol2_df['author'].apply(fix_author)
vol3_df['author'] = vol3_df['author'].apply(fix_author)

#changing name of columns 
vol1_df.rename(columns={'genus': 'mouterde_genus', 'epithet': 'mouterde_epithet', 'author':'mouterde_author', 'infra':'mouterde_infra'}, inplace=True)
vol2_df.rename(columns={'genus': 'mouterde_genus', 'epithet': 'mouterde_epithet', 'author':'mouterde_author', 'infra':'mouterde_infra'}, inplace=True)
vol3_df.rename(columns={'genus': 'mouterde_genus', 'epithet': 'mouterde_epithet', 'author':'mouterde_author', 'infra':'mouterde_infra'}, inplace=True)

#wcvp 
text_wcvp_path = '../input/input copy/wcvp/wcvp_v9_jun_2022.txt'
wcvp_df = pd.read_csv(text_wcvp_path, sep = '|')

In [105]:
vol1_df['mouterde_author']

,page_num,mouterde_genus,mouterde_epithet,mouterde_infra,mouterde_author,taxon rank
0,616,Acanthophyllum,NaN,NaN,NaN,genus
1,616,Acanthophyllum,kurdicum,NaN,Boiss. et Hausskn.,species
2,616,Achyranthes,NaN,NaN,NaN,genus
3,616,Achyranthes,aspera,NaN,L.,species
4,616,Adiantum,NaN,NaN,NaN,genus
...,...,...,...,...,...,...
1382,638,Vulpia,uniglumis,NaN,(Sol.) Dumort.,species
1383,638,Zannichellia,NaN,NaN,NaN,genus
1384,638,Zannichellia,palustris,NaN,L.,species
1385,638,Zostera,NaN,NaN,NaN,genus


#### regex part 

regex for close pranthesis: 
- .+\)
    - split the string into in and out of paranthesis
regex for anything that comes after ex (with two space around it):
- \sex[\s|.].*
    - remove everything after ex (in each set created by end paranthesis)
- \set[\s|.]|\sin[\s|.]|\snon[\s|.]
    - split each set (in each set created by end paranthesis) by et, in, non after that

In [356]:
#using regex above
def split_author(row):
    if (not isinstance(row, str)) or row == "np.NaN":
        return row
    paran_list = []
    reg_list = []

    in_pran = ""
    out_pran = ""
    #the paranthesis part
    search_full_paran = re.search(r"\(.*\)", row)
    search_close_paran = re.search(r".*\)", row)
    if search_full_paran:
        in_pran = search_full_paran.group(0)[1:-1] #assuming we have at most 1 part of () prob use sub later
    elif search_close_paran:
        in_pran = search_full_paran.group(0)[:-1]
    
    split_paran = re.split(r".*\)", row, maxsplit=1) #not sure if this removes anything after last or not 
    out_pran = split_paran[int(len(split_paran) == 2)]
    
    #remove anything after ex. in out_pran and in_pran
    in_pran = re.sub(r"\sex[\s|.].*","",in_pran)
    out_pran = re.sub(r"\sex[\s|.].*","",out_pran)

    #split by et, in non with possible . after them
    #ignoring et, in, non, &, er, nec, with potential . after 
    ignore_pattern = r"\set[\s|.]|\sin[\s|.]|\snon[\s|.]|\s&[\s|.]|\ser[\s|.]|\snec[\s|.]|\smult[\s|.]"
    paran_list = re.split(r"\set[\s|.]|\sin[\s|.]|\snon[\s|.]|\s&[\s|.]", in_pran) #WILL NOT REMOVE IF et AT END
    reg_list = re.split(r"\set[\s|.]|\sin[\s|.]|\snon[\s|.]|\s&[\s|.]", out_pran) 
    
    return [aut for aut in paran_list + reg_list if aut != ""] #bandaid lol but it's still very fast so doesn't matter really

In [214]:
vol3_df['mouterde_author_list'] = vol3_df['mouterde_author'].apply(split_author)

In [215]:
vol3_df.tail(50)

,page_num,mouterde_genus,mouterde_epithet,mouterde_infra,mouterde_author,taxon rank,mouterde_author_list
1572,581,Veronica,panormitana,Fischer,NaN,A.,NaN
1573,581,Veronica,panormitana,panormitana,NaN,subsp.,NaN
1574,581,Veronica,persica,NaN,Poir.,species,[Poir.]
1575,581,Veronica,polifolia,NaN,Benth.,species,[Benth.]
1576,581,Veronica,poliţa,NaN,Fries,species,[Fries]
1577,581,Veronica,pusilla,NaN,Kotschy et Boiss.,species,"[Kotschy, Boiss.]"
1578,581,Veronica,reuterana,NaN,Boiss.,species,[Boiss.]
1579,581,Veronica,samuelssonii,NaN,Rech.,species,[Rech.]
1580,581,Veronica,scardica,NaN,Griseb. lysimachioides Boiss.,species,[Griseb. lysimachioides Boiss.]
1581,581,Veronica,syriaca,NaN,Roem. et Sch.,species,"[Roem., Sch.]"


### breaking down wcvp authors list like mout.

#### feb 19

tldr : need to update wcvp author set now to work similar to the mout authors in regex
- added & to account for variations (et vs &). ex works as expected. Not sure about other symbols tho ... 

In [219]:
wcvp_df['author_list'] = wcvp_df['authors'].apply(split_author)

In [218]:
wcvp_df.head(50)

,kew_id,family,genus,species,infraspecies,taxon_name,authors,rank,taxonomic_status,accepted_kew_id,accepted_name,accepted_authors,parent_kew_id,parent_name,parent_authors,reviewed,publication,original_name_id,authors_set,author_list
0,338-1,Acanthaceae,Acanthodium,np.NaN,np.NaN,Acanthodium,Delile,GENUS,Synonym,427-1,Blepharis,Juss.,NaN,NaN,NaN,In review,"Descr. Egypte, Hist. Nat. 2(Mém.): 241 (1813)",NaN,{Delile},[Delile]
1,44787-1,Acanthaceae,Acanthodium,angustum,np.NaN,Acanthodium angustum,Nees,SPECIES,Homotypic_Synonym,46469-1,Blepharis angusta,(Nees) T.Anderson,NaN,NaN,NaN,In review,"A.P.de Candolle, Prodr. 11: 273 (1847)",NaN,{Nees},[Nees]
2,44788-1,Acanthaceae,Acanthodium,capense,np.NaN,Acanthodium capense,(L.f.) Nees,SPECIES,Homotypic_Synonym,46487-1,Blepharis capensis,(L.f.) Pers.,NaN,NaN,NaN,In review,Linnaea 15: 361 (1841),NaN,"{Nees, (L.f.)}","[L.f., Nees]"
3,44789-1,Acanthaceae,Acanthodium,carduifolium,np.NaN,Acanthodium carduifolium,(L.f.) Nees,SPECIES,Homotypic_Synonym,44830-1,Acanthopsis carduifolia,(L.f.) Schinz,NaN,NaN,NaN,In review,"A.P.de Candolle, Prodr. 11: 278 (1847)",44848-1,"{Nees, (L.f.)}","[L.f., Nees]"
4,44790-1,Acanthaceae,Acanthodium,delilii,np.NaN,Acanthodium delilii,H.Buek,SPECIES,Synonym,46503-1,Blepharis edulis,(Forssk.) Pers.,NaN,NaN,NaN,In review,Gen. Sp. Candoll. 3: 1 (1858),NaN,{H.Buek},[H.Buek]
5,44792-1,Acanthaceae,Acanthodium,diversispinum,np.NaN,Acanthodium diversispinum,Nees,SPECIES,Homotypic_Synonym,46501-1,Blepharis diversispina,(Nees) C.B.Clarke,NaN,NaN,NaN,In review,"A.P.de Candolle, Prodr. 11: 275 (1847)",NaN,{Nees},[Nees]
6,44793-1,Acanthaceae,Acanthodium,furcatum,np.NaN,Acanthodium furcatum,(L.f.) Nees,SPECIES,Homotypic_Synonym,46514-1,Blepharis furcata,(L.f.) Pers.,NaN,NaN,NaN,In review,"A.P.de Candolle, Prodr. 11: 276 (1847)",44862-1,"{Nees, (L.f.)}","[L.f., Nees]"
7,44794-1,Acanthaceae,Acanthodium,glabrum,np.NaN,Acanthodium glabrum,Nees,SPECIES,Homotypic_Synonym,77177568-1,Acanthopsis glabra,(Nees) H.M.Steyn,NaN,NaN,NaN,In review,"A.P.de Candolle, Prodr. 11: 278 (1847)",NaN,{Nees},[Nees]
8,44795-1,Acanthaceae,Acanthodium,glaucum,np.NaN,Acanthodium glaucum,Nees,SPECIES,Homotypic_Synonym,44832-1,Acanthopsis glauca,(Nees) Schinz,NaN,NaN,NaN,In review,"A.P.de Candolle, Prodr. 11: 277 (1847)",NaN,{Nees},[Nees]
9,44796-1,Acanthaceae,Acanthodium,grossum,np.NaN,Acanthodium grossum,Wight,SPECIES,Synonym,46587-1,Blepharis scindica,Stocks ex T.Anderson,NaN,NaN,NaN,In review,Icon. Pl. Ind. Orient. 4: t. 1535 (1850),NaN,{Wight},[Wight]


In [209]:
wcvp_df_explode[wcvp_df_explode['genus'] == 'Ziziphora']

,kew_id,family,genus,species,infraspecies,taxon_name,authors,rank,taxonomic_status,accepted_kew_id,accepted_name,accepted_authors,parent_kew_id,parent_name,parent_authors,reviewed,publication,original_name_id,authors_set,authors_set_
667550,21306-1,Lamiaceae,Ziziphora,np.NaN,np.NaN,Ziziphora,L.,GENUS,Accepted,NaN,NaN,NaN,NaN,NaN,NaN,Reviewed,Sp. Pl.: 21 (1753),NaN,L.,L.
667551,461958-1,Lamiaceae,Ziziphora,abd-el-asisii,np.NaN,Ziziphora abd-el-asisii,Hand.-Mazz.,SPECIES,Synonym,77170162-1,Ziziphora taurica subsp. taurica,NaN,NaN,NaN,NaN,Reviewed,Ann. K. K. Naturhist. Hofmus. 27: 418 (1913),NaN,Hand.-Mazz.,Hand.-Mazz.
667552,77248417-1,Lamiaceae,Ziziphora,acinoides,np.NaN,Ziziphora acinoides,Willk. & Lange,SPECIES,Synonym,60468599-2,Ziziphora aragonensis,Pau,NaN,NaN,NaN,Reviewed,Prodr. Fl. Hispan. 2: 428 (1868),NaN,Lange,Lange
667552,77248417-1,Lamiaceae,Ziziphora,acinoides,np.NaN,Ziziphora acinoides,Willk. & Lange,SPECIES,Synonym,60468599-2,Ziziphora aragonensis,Pau,NaN,NaN,NaN,Reviewed,Prodr. Fl. Hispan. 2: 428 (1868),NaN,Willk.,Willk.
667553,461960-1,Lamiaceae,Ziziphora,acinoides,np.NaN,Ziziphora acinoides,Pall.,SPECIES,Synonym,77171296-1,Ziziphora clinopodioides subsp. clinopodioides,NaN,NaN,NaN,NaN,Reviewed,Reise Russ. Reich. 3: 362 (1776),NaN,Pall.,Pall.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667666,934226-1,Lamiaceae,Ziziphora,vichodceviana,np.NaN,Ziziphora vichodceviana,Tkatsch. ex Tulyag.,SPECIES,Accepted,NaN,NaN,NaN,21306-1,Ziziphora,L.,Reviewed,Opred. Rast. Sred. Azii 9: 362 (1987),NaN,Tkatsch.,Tkatsch.
667666,934226-1,Lamiaceae,Ziziphora,vichodceviana,np.NaN,Ziziphora vichodceviana,Tkatsch. ex Tulyag.,SPECIES,Accepted,NaN,NaN,NaN,21306-1,Ziziphora,L.,Reviewed,Opred. Rast. Sred. Azii 9: 362 (1987),NaN,Tulyag.,Tulyag.
667667,77165291-1,Lamiaceae,Ziziphora,villosa,np.NaN,Ziziphora villosa,(Pers.) Melnikov,SPECIES,Synonym,445709-1,Clinopodium acinos,(L.) Kuntze,NaN,NaN,NaN,Reviewed,Bot. Zhurn. (Moscow & Leningrad) 101: 89 (2016),444245-1,(Pers.),Pers.
667667,77165291-1,Lamiaceae,Ziziphora,villosa,np.NaN,Ziziphora villosa,(Pers.) Melnikov,SPECIES,Synonym,445709-1,Clinopodium acinos,(L.) Kuntze,NaN,NaN,NaN,Reviewed,Bot. Zhurn. (Moscow & Leningrad) 101: 89 (2016),444245-1,Melnikov,Melnikov


### Get Closest Match

instead of having genus == other genus could have

- fuzzy matching with that genus is above 85% 
- then get the max of such a match 

In [220]:
def is_potential_match(string1, string2, threshold = 85):
    # WRatio used in process.extract
    if fuzz.WRatio(string1, string2) > threshold:
        return True
    return False 

In [358]:
mouterde_df = vol3_df
wcvp_df 

merge_df = pd.merge(mouterde_df_explode, 
                        wcvp_df_explode, 
                        how='inner', 
                        left_on=['mouterde_genus','mouterde_epithet', 'mouterde_infra' , 'authors_set'], 
                        right_on = ['genus','species', 'infraspecies', 'authors_set'])

merge_df_simplified = merge_df[['mouterde_genus', 'mouterde_epithet', 'mouterde_infra', 'mouterde_author', 'authors', 'taxonomic_status', 'accepted_name', 'accepted_authors']]
merge_df_simplified.drop_duplicates()

KeyError: 'authors_set'

In [235]:
def fuzzy_merge(df_1, df_2, key1, key2, threshold=85, limit=5):
    """
    :param df_1: the left table to join
    :param df_2: the right table to join
    :param key1: key column of the left table
    :param key2: key column of the right table
    :param threshold: how close the matches should be to return a match, based on Levenshtein distance
    :param limit: the amount of matches that will get returned, these are sorted high to low
    :return: dataframe with boths keys and matches
    """
    s = df_2[key2].tolist()
    
    m = df_1[key1].apply(lambda x: process.extract(x, s, limit=limit))    
    df_1['matches'] = m
    
    m2 = df_1['matches'].apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
    df_1['matches'] = m2
    
    return df_1

In [237]:
fuzzy_merge(vol3_df, wcvp_df, "mouterde_genus", "genus")

KeyboardInterrupt: 

In [238]:
a = "abd-el-asisii"
b = "'Abd-el-'asissi"
fuzz.WRatio(a, b)

89

In [239]:
import difflib

1232931

In [245]:
vol3_df['mout_genus_close_match'] = vol3_df['mouterde_genus'].apply(lambda x: difflib.get_close_matches(x, wcvp_df['genus'].unique())[0])

In [255]:
vol3_df[vol3_df['mout_genus_close_match'] != vol3_df['mouterde_genus']].head(60)[["mout_genus_close_match", "mouterde_genus", "mouterde_epithet", "mouterde_infra", "mouterde_author"]]

,mout_genus_close_match,mouterde_genus,mouterde_epithet,mouterde_infra,mouterde_author
71,Anthemis,Anthémis,NaN,NaN,L.
72,Anthemis,Anthémis,aaronsohnii,NaN,Eig
73,Anthemis,Anthémis,altissima,NaN,L.
74,Anthemis,Anthémis,austriaca,NaN,Jacq.
75,Anthemis,Anthémis,breviradiata,NaN,Eig
76,Anthemis,Anthémis,chia,NaN,L. cornucopiae Boiss.
77,Corymbula,corymbulosa,NaN,NaN,Boiss.
78,Rotula,cotula,NaN,NaN,L.
79,Herpetica,eretica,NaN,NaN,L.
80,Herpetica,eretica,subsp.,NaN,cassia (Boiss.) Grierson


In [257]:
wcvp_df['species'].unique()[0]

'np.NaN'

In [258]:
vol3_df['mouterde_epithet'].replace(np.nan, 'np.NaN', inplace=True)

In [259]:
vol3_df['mout_epithet_close_match'] = vol3_df['mouterde_epithet'].apply(lambda x: difflib.get_close_matches(x, wcvp_df['species'].unique())[0])

I am absolutely confused about the parameters that I should do fuzzy matching on here aaa

In [263]:
vol3_df[vol3_df['mout_epithet_close_match'] != vol3_df['mouterde_epithet']][["mout_genus_close_match", "mouterde_genus", "mout_epithet_close_match","mouterde_epithet", "mouterde_author"]]

,mout_genus_close_match,mouterde_genus,mout_epithet_close_match,mouterde_epithet,mouterde_author
80,Herpetica,eretica,subasper,subsp.,cassia (Boiss.) Grierson
81,Herpetica,eretica,varus,var.,discoidea Post
97,Pseudocunila,pseudocotula,varus,var.,urvilleana DC.
99,Pseudocunila,pseudocotula,varus,var.,discoidea (AH.) Vahl
126,Artemisia,Artemisia,verlotiorum,verlotorum,Lamotte
320,Centaurea,Centaurea,ptosimopappa,ptosimoppapa,Hayek
347,Centranthus,Centranthus,longiflorus,longinorus,Stev.
348,Centranthus,Centranthus,longiflorus,longinorus,Boiss.
462,Crepis,Crépis,zazyntha,zacyntha,(L.) Babcock
464,Cressa,Cressa,ereica,eretica,L.


##### outlineing fuzzy matching for authors only

In [ ]:
authors_set_src = []
authors_set_dst = []
num_src = 0
num_dst = 0 
for author_src in authors_set_src:
    for author_dst in authors_set_dst:
        if is_potential_match(author_src, author_dst):
            #also find a way to chceck it's a best match so for example apply is potentail match to all elements
            #this is essentially using process.extract tho ... hmmmm
            num_src += 1
            num_dst += 1

if num_src == len(authors_set_src) and len(authors_set_src) == len(authors_set_dst):
    "type 0"
if num_src == len(authors_set_src) and len(authors_set_src) < len(authors_set_dst):
    "type 1"
if num_src == len(authors_set_src) and len(authors_set_src) > len(authors_set_dst):
    "type 2" 

if num_src < len(authors_set_dst) and len(authors_set_src) > len(authors_set_dst):
    "type x"

if num_src < len(authors_set_dst) and len(authors_set_src) < len(authors_set_dst):
    "type y"


"""etc etc"""

In [265]:
vol3_df

,page_num,mouterde_genus,mouterde_epithet,mouterde_infra,mouterde_author,taxon rank,mouterde_author_list,mout_genus_close_match,mout_epithet_close_match
0,555,Acantholepis,np.NaN,NaN,Less.',genus,[Less.'],Acantholepis,np.NaN
1,555,Acantholepis,orientalis,NaN,Less.,species,[Less.],Acantholepis,orientalis
2,555,Acantholimon,np.NaN,NaN,Boiss.,genus,[Boiss.],Acantholimon,np.NaN
3,555,Acantholimon,acerosum,NaN,(Willd.) Boiss.,species,"[Willd., Boiss.]",Acantholimon,acerosum
4,555,Acantholimon,antilibanoticum,NaN,Mout.,species,[Mout.],Acantholimon,antilibanoticum
...,...,...,...,...,...,...,...,...,...
1617,582,Ziziphora,taurica,NaN,Bieb.,species,[Bieb.],Ziziphora,taurica
1618,582,Ziziphora,tenuior,NaN,L.,species,[L.],Ziziphora,tenuior
1619,582,Zoegea,np.NaN,NaN,L.,genus,[L.],Zoegea,np.NaN
1620,582,Zoegea,leptaurea,NaN,L.,species,[L.],Zoegea,leptaurea


In [266]:
wcvp_df

,kew_id,family,genus,species,infraspecies,taxon_name,authors,rank,taxonomic_status,accepted_kew_id,accepted_name,accepted_authors,parent_kew_id,parent_name,parent_authors,reviewed,publication,original_name_id,authors_set,author_list
0,338-1,Acanthaceae,Acanthodium,np.NaN,np.NaN,Acanthodium,Delile,GENUS,Synonym,427-1,Blepharis,Juss.,NaN,NaN,NaN,In review,"Descr. Egypte, Hist. Nat. 2(Mém.): 241 (1813)",NaN,{Delile},[Delile]
1,44787-1,Acanthaceae,Acanthodium,angustum,np.NaN,Acanthodium angustum,Nees,SPECIES,Homotypic_Synonym,46469-1,Blepharis angusta,(Nees) T.Anderson,NaN,NaN,NaN,In review,"A.P.de Candolle, Prodr. 11: 273 (1847)",NaN,{Nees},[Nees]
2,44788-1,Acanthaceae,Acanthodium,capense,np.NaN,Acanthodium capense,(L.f.) Nees,SPECIES,Homotypic_Synonym,46487-1,Blepharis capensis,(L.f.) Pers.,NaN,NaN,NaN,In review,Linnaea 15: 361 (1841),NaN,"{Nees, (L.f.)}","[L.f., Nees]"
3,44789-1,Acanthaceae,Acanthodium,carduifolium,np.NaN,Acanthodium carduifolium,(L.f.) Nees,SPECIES,Homotypic_Synonym,44830-1,Acanthopsis carduifolia,(L.f.) Schinz,NaN,NaN,NaN,In review,"A.P.de Candolle, Prodr. 11: 278 (1847)",44848-1,"{Nees, (L.f.)}","[L.f., Nees]"
4,44790-1,Acanthaceae,Acanthodium,delilii,np.NaN,Acanthodium delilii,H.Buek,SPECIES,Synonym,46503-1,Blepharis edulis,(Forssk.) Pers.,NaN,NaN,NaN,In review,Gen. Sp. Candoll. 3: 1 (1858),NaN,{H.Buek},[H.Buek]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1232926,873721-1,Zygophyllaceae,Zygophyllum,webbianum,np.NaN,Zygophyllum webbianum,Coss.,SPECIES,Synonym,873565-1,Zygophyllum fontanesii,Webb & Berthel.,NaN,NaN,NaN,In review,Bull. Soc. Bot. France 2: 365 (1855),NaN,{Coss.},[Coss.]
1232927,77205534-1,Zygophyllaceae,Zygophyllum,xanthoxylum,np.NaN,Zygophyllum xanthoxylum,(Bunge) Maxim.,SPECIES,Accepted,NaN,NaN,NaN,41749-1,Zygophyllum,L.,In review,Fl. Tangut.: 103 (1889),873370-1,"{Maxim., (Bunge)}","[Bunge, Maxim.]"
1232928,873723-1,Zygophyllaceae,Zygophyllum,zanthoxylum,np.NaN,Zygophyllum zanthoxylum,Engl. ex Dippel,SPECIES,Synonym,77205534-1,Zygophyllum xanthoxylum,(Bunge) Maxim.,NaN,NaN,NaN,In review,Handb. Laubholzk. 2: 358 (1893),NaN,"{Engl., Dippel}",[Engl.]
1232929,77184991-1,Zygophyllaceae,Zygophyllum,zilloides,np.NaN,Zygophyllum zilloides,(Humbert) Christenh. & Byng,SPECIES,Accepted,NaN,NaN,NaN,41749-1,Zygophyllum,L.,In review,Global Fl. 4: 93 (2018),873235-1,"{Christenh., Byng, (Humbert)}","[Humbert, Christenh., Byng]"


In [268]:
mouterde_df = vol3_df
mouterde_df['mouterde_infra'].replace(np.nan, 'np.NaN', inplace=True)
mouterde_df['mouterde_epithet'].replace(np.nan, 'np.NaN', inplace=True)
#mouterde_df['mouterde_author'] = mouterde_df['mouterde_author'].str.replace(' et ', ' & ')

#mouterde_df['authors_set'] = mouterde_df['mouterde_author'].str.split(' ').apply(lambda x : get_set(x) - connectors)
#mouterde_df_explode = mouterde_df.explode('authors_set')
#mouterde_df_explode['authors_set'] = mouterde_df_explode['authors_set'].apply(remove_prantheses) #to remove prantheses

# had duplicates 
merge_df = pd.merge(mouterde_df, 
                    wcvp_df, 
                    how='inner', 
                    left_on=['mouterde_genus','mouterde_epithet', 'mouterde_infra'], 
                    right_on = ['genus','species', 'infraspecies'])

merge_df_simplified = merge_df[['mouterde_genus', 
                                'mouterde_epithet', 
                                'mouterde_infra', 
                                'mouterde_author', 
                                'mouterde_author_list', 
                                'author_list', 
                                'authors', 
                                'taxonomic_status', 
                                'accepted_name', 
                                'accepted_authors']]

In [406]:
merge_df = pd.merge(mouterde_df, 
                    wcvp_df, 
                    how='inner', 
                    left_on=['mouterde_genus','mouterde_epithet', 'mouterde_infra'], 
                    right_on = ['genus','species', 'infraspecies'])

merge_df_simplified = merge_df[['mouterde_genus', 
                                'mouterde_epithet', 
                                'mouterde_infra', 
                                'mouterde_author', 
                                'mouterde_author_list', 
                                'author_list', 
                                'authors', 
                                'genus',
                                'species'
                                ]]

merge_df_simplified_copy = merge_df_simplified.copy()

In [408]:
merge_df_simplified[(merge_df_simplified['genus'] == "Xeranthemum") & (merge_df_simplified['species'] == "cylindraceum")]

,mouterde_genus,mouterde_epithet,mouterde_infra,mouterde_author,mouterde_author_list,author_list,authors,genus,species
1588,Xeranthemum,cylindraceum,np.NaN,Sm. in Sibth. et Sm.,"[Sm., Sibth., Sm.]",[Sm.],Sm.,Xeranthemum,cylindraceum
1589,Xeranthemum,cylindraceum,np.NaN,Sm. in Sibth. et Sm.,"[Sm., Sibth., Sm.]",[Colmeiro],Colmeiro,Xeranthemum,cylindraceum


In [409]:
merge_df_simplified

,mouterde_genus,mouterde_epithet,mouterde_infra,mouterde_author,mouterde_author_list,author_list,authors,genus,species
0,Acantholepis,np.NaN,np.NaN,Less.',[Less.'],[Less.],Less.,Acantholepis,np.NaN
1,Acantholepis,orientalis,np.NaN,Less.,[Less.],[Less.],Less.,Acantholepis,orientalis
2,Acantholimon,np.NaN,np.NaN,Boiss.,[Boiss.],[Boiss.],Boiss.,Acantholimon,np.NaN
3,Acantholimon,acerosum,np.NaN,(Willd.) Boiss.,"[Willd., Boiss.]","[Willd., Boiss.]",(Willd.) Boiss.,Acantholimon,acerosum
4,Acantholimon,antilibanoticum,np.NaN,Mout.,[Mout.],[Mouterde],Mouterde,Acantholimon,antilibanoticum
...,...,...,...,...,...,...,...,...,...
1597,Ziziphora,taurica,np.NaN,Bieb.,[Bieb.],[M.Bieb.],M.Bieb.,Ziziphora,taurica
1598,Ziziphora,tenuior,np.NaN,L.,[L.],[L.],L.,Ziziphora,tenuior
1599,Zoegea,np.NaN,np.NaN,L.,[L.],[L.],L.,Zoegea,np.NaN
1600,Zoegea,leptaurea,np.NaN,L.,[L.],[L.],L.,Zoegea,leptaurea


In [410]:
merge_df_simplified[merge_df_simplified['mouterde_genus'] == "Acantholimon"]

,mouterde_genus,mouterde_epithet,mouterde_infra,mouterde_author,mouterde_author_list,author_list,authors,genus,species
2,Acantholimon,np.NaN,np.NaN,Boiss.,[Boiss.],[Boiss.],Boiss.,Acantholimon,np.NaN
3,Acantholimon,acerosum,np.NaN,(Willd.) Boiss.,"[Willd., Boiss.]","[Willd., Boiss.]",(Willd.) Boiss.,Acantholimon,acerosum
4,Acantholimon,antilibanoticum,np.NaN,Mout.,[Mout.],[Mouterde],Mouterde,Acantholimon,antilibanoticum
5,Acantholimon,damassanum,np.NaN,Mobayen,[Mobayen],[Mobayen],Mobayen,Acantholimon,damassanum
6,Acantholimon,libanoticum,np.NaN,Boiss.,[Boiss.],[Boiss.],Boiss.,Acantholimon,libanoticum
7,Acantholimon,ulicinum,np.NaN,(Schultes) Boiss.,"[Schultes, Boiss.]","[Willd., Boiss.]",(Willd. ex Schult.) Boiss.,Acantholimon,ulicinum


In [411]:
wcvp_df[wcvp_df['genus'] == "Achillea"]

,kew_id,family,genus,species,infraspecies,taxon_name,authors,rank,taxonomic_status,accepted_kew_id,accepted_name,accepted_authors,parent_kew_id,parent_name,parent_authors,reviewed,publication,original_name_id,authors_set,author_list
149506,60436826-2,Asteraceae,Achillea,np.NaN,np.NaN,Achillea,L.,GENUS,Accepted,NaN,NaN,NaN,NaN,NaN,NaN,In review,Sp. Pl.: 896 (1753),NaN,{L.},[L.]
149507,173777-1,Asteraceae,Achillea,abrotanifolia,np.NaN,Achillea abrotanifolia,L.,SPECIES,Synonym,252207-1,Tanacetum abrotanifolium,Druce,NaN,NaN,NaN,In review,Sp. Pl.: 897 (1753),NaN,{L.},[L.]
149508,173778-1,Asteraceae,Achillea,abrotanifolia,np.NaN,Achillea abrotanifolia,Steud.,SPECIES,Synonym,952514-1,Tanacetum polycephalum subsp. argyrophyllum,(K.Koch) Podlech,NaN,NaN,NaN,In review,"Nomencl. Bot., ed. 2, 1: 12 (1840)",NaN,{Steud.},[Steud.]
149509,173779-1,Asteraceae,Achillea,abrotanoides,np.NaN,Achillea abrotanoides,(Vis.) Vis.,SPECIES,Accepted,NaN,NaN,NaN,60436826-2,Achillea,L.,In review,Fl. Dalmat. 2: 81 (1847),240054-1,"{Vis., (Vis.)}","[Vis., Vis.]"
149510,173780-1,Asteraceae,Achillea,abscondita,np.NaN,Achillea × abscondita,Wein,SPECIES,Accepted,NaN,NaN,NaN,60436826-2,Achillea,L.,In review,Repert. Spec. Nov. Regni Veg. 11: 395 (1912),NaN,{Wein},[Wein]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150323,174355-1,Asteraceae,Achillea,wilhelmsii,np.NaN,Achillea wilhelmsii,K.Koch,SPECIES,Accepted,NaN,NaN,NaN,60436826-2,Achillea,L.,In review,Linnaea 24: 328 (1851),NaN,{K.Koch},[K.Koch]
150324,1004822-1,Asteraceae,Achillea,wilhelmsii,santolinoides,Achillea wilhelmsii subsp. santolinoides,(Lag.) Vetschera & Kästner,SUBSPECIES,Homotypic_Synonym,174241-1,Achillea santolinoides,Lag.,NaN,NaN,NaN,In review,Feddes Repert. 109: 504 (1998),174241-1,"{Kästner, (Lag.), Vetschera}","[Lag., Vetschera, Kästner]"
150325,174356-1,Asteraceae,Achillea,wilsoniana,np.NaN,Achillea wilsoniana,(Heimerl) Hand.-Mazz.,SPECIES,Accepted,NaN,NaN,NaN,60436826-2,Achillea,L.,In review,Symb. Sin. 7: 1110 (1936),NaN,"{Hand.-Mazz., (Heimerl)}","[Heimerl, Hand.-Mazz.]"
150326,174357-1,Asteraceae,Achillea,wolffii,np.NaN,Achillea wolffii,Schur ex Nyman,SPECIES,Synonym,174032-1,Achillea impatiens,L.,NaN,NaN,NaN,In review,Consp. Fl. Eur.: 365 (1879),NaN,"{Nyman, Schur}",[Schur]


In [412]:
merge_df_simplified.head(30)

,mouterde_genus,mouterde_epithet,mouterde_infra,mouterde_author,mouterde_author_list,author_list,authors,genus,species
0,Acantholepis,np.NaN,np.NaN,Less.',[Less.'],[Less.],Less.,Acantholepis,np.NaN
1,Acantholepis,orientalis,np.NaN,Less.,[Less.],[Less.],Less.,Acantholepis,orientalis
2,Acantholimon,np.NaN,np.NaN,Boiss.,[Boiss.],[Boiss.],Boiss.,Acantholimon,np.NaN
3,Acantholimon,acerosum,np.NaN,(Willd.) Boiss.,"[Willd., Boiss.]","[Willd., Boiss.]",(Willd.) Boiss.,Acantholimon,acerosum
4,Acantholimon,antilibanoticum,np.NaN,Mout.,[Mout.],[Mouterde],Mouterde,Acantholimon,antilibanoticum
5,Acantholimon,damassanum,np.NaN,Mobayen,[Mobayen],[Mobayen],Mobayen,Acantholimon,damassanum
6,Acantholimon,libanoticum,np.NaN,Boiss.,[Boiss.],[Boiss.],Boiss.,Acantholimon,libanoticum
7,Acantholimon,ulicinum,np.NaN,(Schultes) Boiss.,"[Schultes, Boiss.]","[Willd., Boiss.]",(Willd. ex Schult.) Boiss.,Acantholimon,ulicinum
8,Acanthus,np.NaN,np.NaN,L.,[L.],[L.],L.,Acanthus,np.NaN
9,Acanthus,dioscoridis,np.NaN,L.,[L.],[L.],L.,Acanthus,dioscoridis


In [413]:
def get_match_type(row):
    authors_set_src = set(row['mouterde_author_list'])
    authors_set_dst = set(row['author_list'])
    num_src = 0
    for author in authors_set_src: 
        _, score = process.extract(author, authors_set_dst)[0]
        if score > 85:
            num_src += 1
    type = "NO TYPE"
    if num_src == 0 and len(authors_set_src) > 0 and  len(authors_set_dst) > 0:
        type = "type -1"
    elif num_src == len(authors_set_src) and len(authors_set_src) == len(authors_set_dst):
        type = "type 0" #everything the same
    elif num_src == len(authors_set_src) and len(authors_set_src) < len(authors_set_dst):
        type = "type 1" #everything in src match but dst set has more 
    elif num_src == len(authors_set_dst) and len(authors_set_src) > len(authors_set_dst):
        type = "type 2" #everything in dst match but src has more 

    elif num_src < min(len(authors_set_src), len(authors_set_dst)):
        type = "type 3" # a subset of dst or src do not overlap (and one is not a proper subset of another)
    else:
        print(num_src, authors_set_src, authors_set_dst)
    return type

In [414]:
def get_num_overlap(row):
    authors_set_src = set(row['mouterde_author_list'])
    authors_set_dst = set(row['author_list'])
    num_src = 0
    for author in authors_set_src: 
        _, score = process.extract(author, authors_set_dst)[0]
        if score > 85:
            num_src += 1
    return num_src

In [415]:
merge_df_simplified

,mouterde_genus,mouterde_epithet,mouterde_infra,mouterde_author,mouterde_author_list,author_list,authors,genus,species
0,Acantholepis,np.NaN,np.NaN,Less.',[Less.'],[Less.],Less.,Acantholepis,np.NaN
1,Acantholepis,orientalis,np.NaN,Less.,[Less.],[Less.],Less.,Acantholepis,orientalis
2,Acantholimon,np.NaN,np.NaN,Boiss.,[Boiss.],[Boiss.],Boiss.,Acantholimon,np.NaN
3,Acantholimon,acerosum,np.NaN,(Willd.) Boiss.,"[Willd., Boiss.]","[Willd., Boiss.]",(Willd.) Boiss.,Acantholimon,acerosum
4,Acantholimon,antilibanoticum,np.NaN,Mout.,[Mout.],[Mouterde],Mouterde,Acantholimon,antilibanoticum
...,...,...,...,...,...,...,...,...,...
1597,Ziziphora,taurica,np.NaN,Bieb.,[Bieb.],[M.Bieb.],M.Bieb.,Ziziphora,taurica
1598,Ziziphora,tenuior,np.NaN,L.,[L.],[L.],L.,Ziziphora,tenuior
1599,Zoegea,np.NaN,np.NaN,L.,[L.],[L.],L.,Zoegea,np.NaN
1600,Zoegea,leptaurea,np.NaN,L.,[L.],[L.],L.,Zoegea,leptaurea


In [416]:
merge_df_simplified.replace(np.nan, 'np.NaN', inplace=True)
merge_df_simplified["author_match_type"] = merge_df_simplified.apply(get_match_type, axis=1)
merge_df_simplified["author_match_score"] = merge_df_simplified.apply(get_num_overlap, axis=1)

/var/folders/gg/rd0htpr953n8c6s3hcyh70kw0000gn/T/ipykernel_41842/353089601.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merge_df_simplified.replace(np.nan, 'np.NaN', inplace=True)
/var/folders/gg/rd0htpr953n8c6s3hcyh70kw0000gn/T/ipykernel_41842/353089601.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merge_df_simplified["author_match_type"] = merge_df_simplified.apply(get_match_type, axis=1)


2 {'p', 'a', '.', 'N', 'n'} {'Ehrend.'}
2 {'L.', ' L.'} {'Lehm.'}
2 {'L.', ' L.'} {'Hablitz'}
2 {'L.', ' L.'} {'Pall.'}


/var/folders/gg/rd0htpr953n8c6s3hcyh70kw0000gn/T/ipykernel_41842/353089601.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merge_df_simplified["author_match_score"] = merge_df_simplified.apply(get_num_overlap, axis=1)


In [417]:
merge_df_simplified = merge_df_simplified.loc[merge_df_simplified.groupby(['mouterde_genus', 'mouterde_epithet', 'mouterde_infra', 'mouterde_author'], sort=False)['author_match_score'].idxmax()]#[['bookid', 'conceptid', 'weight']]


In [418]:
merge_df_simplified[merge_df_simplified['author_match_type'] == 'NO TYPE']

,mouterde_genus,mouterde_epithet,mouterde_infra,mouterde_author,mouterde_author_list,author_list,authors,genus,species,author_match_type,author_match_score
603,Galium,thiebautii,np.NaN,np.NaN,np.NaN,[Ehrend.],Ehrend.,Galium,thiebautii,NO TYPE,2
921,Onosma,orientalis,np.NaN,(L.) L.,"[L., L.]",[Lehm.],Lehm.,Onosma,orientalis,NO TYPE,2


In [419]:
merge_df_simplified[(merge_df_simplified['mouterde_genus'] == 'Onosma') & (merge_df_simplified['mouterde_epithet'] == 'orientalis')]

,mouterde_genus,mouterde_epithet,mouterde_infra,mouterde_author,mouterde_author_list,author_list,authors,genus,species,author_match_type,author_match_score
921,Onosma,orientalis,np.NaN,(L.) L.,"[L., L.]",[Lehm.],Lehm.,Onosma,orientalis,NO TYPE,2
925,Onosma,orientalis,palmyrena,Moût.,[Moût.],[Mouterde],Mouterde ex Charpin & Greuter,Onosma,orientalis,type -1,0


In [420]:
#issue with L. ... good match with everything... 

In [402]:
merge_df_simplified[merge_df_simplified['author_match_type'] == 'type -1']

,mouterde_genus,mouterde_epithet,mouterde_infra,mouterde_author,mouterde_author_list,author_list,authors,genus,species,author_match_type,author_match_score
23,Achillea,wilhelmsii,np.NaN,C. Koch,[C. Koch],[K.Koch],K.Koch,Achillea,wilhelmsii,type -1,0
64,Anchusa,milleri,np.NaN,Willd.,[Willd.],[Lam.],Lam. ex Spreng.,Anchusa,milleri,type -1,0
65,Anchusa,strigosa,np.NaN,Labili.,[Labili.],[Labill.],Labill.,Anchusa,strigosa,type -1,0
116,Asteriscus,np.NaN,np.NaN,Miller,[Miller],[Moench],Moench,Asteriscus,np.NaN,type -1,0
130,Bellardia,np.NaN,np.NaN,Ail.,[Ail.],[Colla],Colla,Bellardia,np.NaN,type -1,0
...,...,...,...,...,...,...,...,...,...,...,...
1506,Verbascum,oreophilum,np.NaN,C. Koch,[C. Koch],[K.Koch],K.Koch,Verbascum,oreophilum,type -1,0
1517,Verbascum,sinuatum,np.NaN,L. tripolitanum Boiss.,[L. tripolitanum Boiss.],[Hablitz],Hablitz,Verbascum,sinuatum,type -1,0
1545,Veronica,macrostachya,macrostachya,orientalis Miller,[orientalis Miller],np.NaN,np.NaN,Veronica,macrostachya,type -1,0
1568,Vinca,major,np.NaN,np.NaN,np.NaN,[Brot.],Brot.,Vinca,major,type -1,0


In [405]:
wcvp_df[(wcvp_df['genus'] == "Anchusa") & (wcvp_df['species'] == "milleri") & (wcvp_df['infraspecies'] == "np.NaN")]

,kew_id,family,genus,species,infraspecies,taxon_name,authors,rank,taxonomic_status,accepted_kew_id,accepted_name,accepted_authors,parent_kew_id,parent_name,parent_authors,reviewed,publication,original_name_id,authors_set,author_list
288711,927538-1,Boraginaceae,Anchusa,milleri,np.NaN,Anchusa milleri,Lam. ex Spreng.,SPECIES,Accepted,NaN,NaN,NaN,4451-1,Anchusa,L.,In review,"Bot. Gart. Halle, Nachtr. 1: 11 (1801)",NaN,"{Lam., Spreng.}",[Lam.]


In [392]:
merge_df_simplified.shape

(1602, 11)

In [394]:
wcvp_df[(wcvp_df['genus'] == "Onosma") & (wcvp_df['species'] == "orientalis") & (wcvp_df['infraspecies'] == "np.NaN")]

,kew_id,family,genus,species,infraspecies,taxon_name,authors,rank,taxonomic_status,accepted_kew_id,accepted_name,accepted_authors,parent_kew_id,parent_name,parent_authors,reviewed,publication,original_name_id,authors_set,author_list
297942,119757-1,Boraginaceae,Onosma,orientalis,np.NaN,Onosma orientalis,Lehm.,SPECIES,Synonym,119658-1,Onosma frutescens,Lam.,NaN,NaN,NaN,In review,Pl. Asperif. Nucif.: 376 (1818),NaN,{Lehm.},[Lehm.]
297943,119754-1,Boraginaceae,Onosma,orientalis,np.NaN,Onosma orientalis,Hablitz,SPECIES,Synonym,119824-1,Onosma stellulata,Waldst. & Kit.,NaN,NaN,NaN,In review,Descr. Phys. Taur.: 116 (1788),NaN,{Hablitz},[Hablitz]
297944,119755-1,Boraginaceae,Onosma,orientalis,np.NaN,Onosma orientalis,(L.) L.,SPECIES,Homotypic_Synonym,120300-1,Podonosma orientalis,(L.) Feinbrun,NaN,NaN,NaN,In review,Sp. Pl. ed. 2: 196 (1762),113828-1,"{(L.), L.}","[L., L.]"
297945,119756-1,Boraginaceae,Onosma,orientalis,np.NaN,Onosma orientalis,Pall.,SPECIES,Synonym,77228219-1,Arnebia decumbens subsp. decumbens,NaN,NaN,NaN,NaN,In review,Reise Russ. Reich. 1: 50 (1771),NaN,{Pall.},[Pall.]


In [331]:
# if there is any overlap -- use world checklist

In [318]:
merge_df_simplified[merge_df_simplified["author_match_type"] == "type 1"]

,mouterde_genus,mouterde_epithet,mouterde_infra,mouterde_author,mouterde_author_list,author_list,authors,taxonomic_status,accepted_name,accepted_authors,author_match_type
88,Armeria,np.NaN,np.NaN,Willd.,[Willd.],"[DC., Willd.]",(DC.) Willd.,Accepted,np.NaN,np.NaN,type 1
128,Atractylis,comosa,np.NaN,Sieb. ex Cass.,[Sieb.],"[Spreng., Sieber]",(Spreng.) Sieber ex Cass.,Homotypic_Synonym,Chamaeleon comosus,(Spreng.) Greuter,type 1
142,Bidens,pilosa,radiata,Schultz Bip.,[Schultz Bip.],"[Sch.Bip., J.A.Schmidt]",(Sch.Bip.) J.A.Schmidt,Synonym,Bidens alba,(L.) DC.,type 1
143,Bidens,pilosa,radiata,Schultz Bip.,[Schultz Bip.],"[Sch.Bip., Pit.]",(Sch.Bip.) Pit.,Synonym,Bidens alba,(L.) DC.,type 1
208,Campanula,pestalozzae,np.NaN,Boiss.,[Boiss.],"[Boiss., Boiss.]",(Boiss.) Boiss.,Synonym,Asyneuma limonifolium subsp. pestalozzae,(Boiss.) Damboldt,type 1
223,Campanula,trachelium,athoa,(Boiss. et Heldr.),"[Boiss., Heldr.]","[Boiss., Heldr., Hayek]",(Boiss. & Heldr.) Hayek,Accepted,np.NaN,np.NaN,type 1
242,Carlina,involucrata,libanotica,(Boiss.) Meusel et,"[Boiss., Meusel et]","[Boiss., Meusel, A.Kástner]",(Boiss.) Meusel & A.Kástner,Synonym,Carlina libanotica,Boiss.,type 1
333,Centaurea,virgata,squarrosa,Boiss.,[Boiss.],"[Boiss., Gugler]",(Boiss.) Gugler,Accepted,np.NaN,np.NaN,type 1
342,Centaurium,maritimum,np.NaN,L.,[L.],"[L., Fritsch]",(L.) Fritsch,Accepted,np.NaN,np.NaN,type 1
417,Convolvulus,stenophyllus,np.NaN,Boiss.,[Boiss.],"[Boiss., Boiss.]",(Boiss.) Boiss.,Synonym,Convolvulus palaestinus,Boiss.,type 1


In [319]:
merge_df_simplified[merge_df_simplified["author_match_type"] == "type 2"]

,mouterde_genus,mouterde_epithet,mouterde_infra,mouterde_author,mouterde_author_list,author_list,authors,taxonomic_status,accepted_name,accepted_authors,author_match_type
21,Achillea,membranacea,np.NaN,(Labili.). DC.,"[Labili., . DC.]",[DC.],DC.,Accepted,np.NaN,np.NaN,type 2
113,Asperula,rivalis,np.NaN,Sibth. et Sm.,"[Sibth., Sm.]",[Sm.],Sm.,Homotypic_Synonym,Galium rivale,(Sm.) Griseb.,type 2
166,Calamintha,incana,np.NaN,(Sibth. et Smith) Boiss.,"[Sibth., Smith, Boiss.]","[Sm., Boiss.]",(Sm.) Boiss.,Synonym,Clinopodium brevifolium,(Wahlenb.) Bräuchler & Hjertson,type 2
167,Calamintha,nepeta,np.NaN,(L.) Savi,"[L., Savi]",[Willk.],Willk.,Synonym,Clinopodium nepeta subsp. spruneri,(Boiss.) Bartolucci & F.Conti,type 2
177,Calamintha,vulgaris,np.NaN,(L.) Halàcsy,"[L., Halàcsy]",[Clairv.],Clairv.,Synonym,Clinopodium nepeta subsp. nepeta,np.NaN,type 2
180,Calamintha,vulgaris,np.NaN,(L.) Halàcsy,"[L., Halàcsy]",[Garsault],Garsault,Synonym,Clinopodium nepeta subsp. spruneri,(Boiss.) Bartolucci & F.Conti,type 2
196,Campanula,cymbalaria,np.NaN,Sm. in Sibth. et Sm.,"[Sm., Sibth., Sm.]",[Sm.],Sm.,Accepted,np.NaN,np.NaN,type 2
200,Campanula,fastigiata,np.NaN,Léon Dufour in A. DC.,"[Léon Dufour, A. DC.]",[Dufour],Dufour ex Schult.,Accepted,np.NaN,np.NaN,type 2
286,Centaurea,cyanoides,np.NaN,Berg et Wahlenb.,"[Berg, Wahlenb.]",[Wahlenb.],Wahlenb.,Accepted,np.NaN,np.NaN,type 2
291,Centaurea,drabifolia,np.NaN,Sibth. et Sm.,"[Sibth., Sm.]",[Sm.],Sm.,Accepted,np.NaN,np.NaN,type 2


In [320]:
merge_df_simplified[merge_df_simplified["author_match_type"] == "type 3"] #use world checklist 

,mouterde_genus,mouterde_epithet,mouterde_infra,mouterde_author,mouterde_author_list,author_list,authors,taxonomic_status,accepted_name,accepted_authors,author_match_type
7,Acantholimon,ulicinum,np.NaN,(Schultes) Boiss.,"[Schultes, Boiss.]","[Willd., Boiss.]",(Willd. ex Schult.) Boiss.,Accepted,np.NaN,np.NaN,type 3
14,Achillea,aleppica,aleppica,np.NaN,np.NaN,np.NaN,np.NaN,Accepted,np.NaN,np.NaN,type 3
61,Anchusa,barrelieri,np.NaN,(AH.) Vitm.,"[AH., Vitm.]","[All., Vitman]",(All.) Vitman,Synonym,Cynoglottis barrelieri subsp. barrelieri,np.NaN,type 3
124,Asyneuma,virgatum,np.NaN,(Labili.) Bornm.,"[Labili., Bornm.]","[Labill., Bornm.]",(Labill.) Bornm.,Accepted,np.NaN,np.NaN,type 3
127,Atractylis,carduus,np.NaN,(Forssk.) Christ,"[Forssk., Christ]","[Forssk., C.Chr.]",(Forssk.) C.Chr.,Accepted,np.NaN,np.NaN,type 3
...,...,...,...,...,...,...,...,...,...,...,...
1549,Veronica,panormitana,baradostensis,(M.A.Fischer) M.,"[M.A.Fischer, M.]","[M.A.Fisch., M.A.Fisch.]",(M.A.Fisch.) M.A.Fisch.,Accepted,np.NaN,np.NaN,type 3
1550,Veronica,panormitana,panormitana,np.NaN,np.NaN,np.NaN,np.NaN,Accepted,np.NaN,np.NaN,type 3
1558,Veronica,triloba,np.NaN,(Opiz) A. Kerner,"[Opiz, A. Kerner]","[Opiz, Opiz]",(Opiz) Opiz,Accepted,np.NaN,np.NaN,type 3
1572,Vincetoxicum,canescens,np.NaN,(Willd.) Deche.,"[Willd., Deche.]","[Willd., Decne.]",(Willd.) Decne.,Accepted,np.NaN,np.NaN,type 3


In [321]:
merge_df_simplified[merge_df_simplified["author_match_type"] == "type -1"]

,mouterde_genus,mouterde_epithet,mouterde_infra,mouterde_author,mouterde_author_list,author_list,authors,taxonomic_status,accepted_name,accepted_authors,author_match_type
23,Achillea,wilhelmsii,np.NaN,C. Koch,[C. Koch],[K.Koch],K.Koch,Accepted,np.NaN,np.NaN,type -1
46,Ambrosia,maritima,np.NaN,L.,[L.],[Ferrero],Ferrero ex DC.,Synonym,Ambrosia hispida,Pursh,type -1
59,Anchusa,azurea,np.NaN,Mill.,[Mill.],[Schur],Schur,Synonym,Anchusa ochroleuca,M.Bieb.,type -1
63,Anchusa,hybrida,np.NaN,Ten.,[Ten.],[Boiss.],Boiss. ex A.DC.,Synonym,Anchusa undulata subsp. granatensis,(Boiss.) Valdés,type -1
64,Anchusa,milleri,np.NaN,Willd.,[Willd.],[Lam.],Lam. ex Spreng.,Accepted,np.NaN,np.NaN,type -1
...,...,...,...,...,...,...,...,...,...,...,...
1569,Vinca,major,np.NaN,np.NaN,np.NaN,[L.],L.,Accepted,np.NaN,np.NaN,type -1
1570,Vincetoxicum,np.NaN,np.NaN,N.M.Wolf,[N.M.Wolf],[Walter],Walter,Synonym,Gonolobus,Michx.,type -1
1589,Xeranthemum,cylindraceum,np.NaN,Sm. in Sibth. et Sm.,"[Sm., Sibth., Sm.]",[Colmeiro],Colmeiro,Synonym,Xeranthemum inapertum,(L.) Mill.,type -1
1590,Xeranthemum,inapertum,np.NaN,(L.) Miller,"[L., Miller]",[M.Bieb.],M.Bieb.,Synonym,Xeranthemum cylindraceum,Sm.,type -1


In [368]:
wcvp_df[(wcvp_df['genus'] == "Xeranthemum") & (wcvp_df['species'] == "cylindraceum")]

,kew_id,family,genus,species,infraspecies,taxon_name,authors,rank,taxonomic_status,accepted_kew_id,accepted_name,accepted_authors,parent_kew_id,parent_name,parent_authors,reviewed,publication,original_name_id,authors_set,author_list
273787,260972-1,Asteraceae,Xeranthemum,cylindraceum,np.NaN,Xeranthemum cylindraceum,Sm.,SPECIES,Accepted,NaN,NaN,NaN,60437401-2,Xeranthemum,L.,In review,"J.Sibthorp & J.E.Smith, Fl. Graec. Prodr. 2: 1...",NaN,{Sm.},[Sm.]
273788,260973-1,Asteraceae,Xeranthemum,cylindraceum,np.NaN,Xeranthemum cylindraceum,Colmeiro,SPECIES,Synonym,260990-1,Xeranthemum inapertum,(L.) Mill.,NaN,NaN,NaN,In review,Apuntes Fl. Castillas: 85 (1849),NaN,{Colmeiro},[Colmeiro]


In [322]:
merge_df_simplified[merge_df_simplified["author_match_type"] == "type 0"]

,mouterde_genus,mouterde_epithet,mouterde_infra,mouterde_author,mouterde_author_list,author_list,authors,taxonomic_status,accepted_name,accepted_authors,author_match_type
0,Acantholepis,np.NaN,np.NaN,Less.',[Less.'],[Less.],Less.,Synonym,Echinops,L.,type 0
1,Acantholepis,orientalis,np.NaN,Less.,[Less.],[Less.],Less.,Homotypic_Synonym,Echinops acantholepis,Jaub. & Spach,type 0
2,Acantholimon,np.NaN,np.NaN,Boiss.,[Boiss.],[Boiss.],Boiss.,Accepted,np.NaN,np.NaN,type 0
3,Acantholimon,acerosum,np.NaN,(Willd.) Boiss.,"[Willd., Boiss.]","[Willd., Boiss.]",(Willd.) Boiss.,Accepted,np.NaN,np.NaN,type 0
4,Acantholimon,antilibanoticum,np.NaN,Mout.,[Mout.],[Mouterde],Mouterde,Accepted,np.NaN,np.NaN,type 0
...,...,...,...,...,...,...,...,...,...,...,...
1597,Ziziphora,taurica,np.NaN,Bieb.,[Bieb.],[M.Bieb.],M.Bieb.,Accepted,np.NaN,np.NaN,type 0
1598,Ziziphora,tenuior,np.NaN,L.,[L.],[L.],L.,Accepted,np.NaN,np.NaN,type 0
1599,Zoegea,np.NaN,np.NaN,L.,[L.],[L.],L.,Accepted,np.NaN,np.NaN,type 0
1600,Zoegea,leptaurea,np.NaN,L.,[L.],[L.],L.,Accepted,np.NaN,np.NaN,type 0


In [323]:
merge_df_simplified[merge_df_simplified["author_match_type"] == "NO TYPE"]

,mouterde_genus,mouterde_epithet,mouterde_infra,mouterde_author,mouterde_author_list,author_list,authors,taxonomic_status,accepted_name,accepted_authors,author_match_type
603,Galium,thiebautii,np.NaN,np.NaN,np.NaN,[Ehrend.],Ehrend.,Accepted,np.NaN,np.NaN,NO TYPE
921,Onosma,orientalis,np.NaN,(L.) L.,"[L., L.]",[Lehm.],Lehm.,Synonym,Onosma frutescens,Lam.,NO TYPE
922,Onosma,orientalis,np.NaN,(L.) L.,"[L., L.]",[Hablitz],Hablitz,Synonym,Onosma stellulata,Waldst. & Kit.,NO TYPE
924,Onosma,orientalis,np.NaN,(L.) L.,"[L., L.]",[Pall.],Pall.,Synonym,Arnebia decumbens subsp. decumbens,np.NaN,NO TYPE


### START Feb 22

In [345]:
author_set = {'test'}
for s in vol3_df['mouterde_author']:
    if isinstance(s, str):
        for w in list(s.split()):
            author_set.add(w)

In [349]:
s = "123$@hi"
print(str(c for c in s if c.isalnum()))

<generator object <genexpr> at 0x2ba6d3120>


In [355]:
author_set_lower_case = set()

for s in author_set:
    pruned_s = ""
    for c in s:
        if c.isalnum():
            pruned_s += c
    if pruned_s.islower():
        author_set_lower_case.add(s)

TypeError: 'dict' object is not callable

In [353]:
author_set_lower_case

{'adoxifolium',
 'agg.',
 'anagallis-aquatica',
 'aperta',
 'asperrimus',
 'assurense',
 'auct.',
 'benedictus',
 'cassia',
 'colocynthis',
 'cornucopiae',
 'coronarium',
 'discoidea',
 'emend.',
 'et',
 'ex',
 'f.',
 'fa',
 'fii.',
 'fil.',
 'fil.)',
 'hispidula',
 'i-à*',
 'in',
 'j.',
 'juncea',
 'kotschyi',
 'lasianthum',
 'leiocarpa',
 'lysimachioides',
 'mult.',
 'nec',
 'non',
 'nty)',
 'orientalis',
 'pentagonia',
 'phelipaea',
 'ptychophyllum',
 's.l.',
 'sinuatum',
 'tenuis',
 'test',
 'tripolitanum',
 'urvilleana',
 'yar.'}

In [ ]:
def find_lower_case(vol):
    author_set = {}
    for s in vol3_df['mouterde_author']:
        if isinstance(s, str):
            for w in list(s.split()):
                author_set.add(w)

### END Feb 22

In [324]:
process.extract("Sm.", ["Sm."])[0]

('Sm.', 100)

##### Potential problem with abreviations
possible fix: dropping voels and running match again?

In [304]:
m_author = "Mout."
wcvp_author = ["(Post)", "Mouterde", "ex" ,"Charpin" ,"&" ,"Greuter"]
process.extract(m_author, wcvp_author)
# can use this, check everything above SCORE 

[('Mouterde', 90), ('(Post)', 50), ('Greuter', 45), ('ex', 0), ('Charpin', 0)]

In [264]:
string1 = "eretica"
string2 = 'cretica'
print(fuzz.WRatio(string1, string2))
is_potential_match(string1, string2)

86


True

## UNTITLED 

### earlier 

In [101]:
def get_closest_match(r):
    m_author = r['mouterde_author'] 
    genus = r['mouterde_genus']
    epithet = r['mouterde_epithet']
    infra = r['mouterde_infra']
    wcvp_df_gs_match = wcvp_df[(wcvp_df['genus'] == genus) & (wcvp_df['species'] == epithet) & (wcvp_df['infraspecies'] == infra)]
    
    possible_author = []

    #this doesn't work as is but here's the idea 
    
    if isinstance(m_author, str):
        m_author_set = list(get_set(m_author.split(' ')) - connectors)
        for author in m_author_set:
            #print(author, process.extract(author, wcvp_df_gs_match['authors_set']))
            try:
                author_match, author_score, _ = process.extract(author, wcvp_df_gs_match['authors_set'])[0]
                print(author, author_match, author_score)
                if author_score >= 85:
                    possible_author.append(author_match)
            except:
                possible_author.append(np.nan)
        #author_match = difflib.get_close_matches(m_author, wcvp_df_gs_match['authors'])
    else:
        possible_author.append(np.nan)
    return possible_author

In [102]:
vol3_df['closest_wcvp_match'] = vol3_df.apply(get_closest_match, axis = 1)

Less.' {'Less.'} 100
Less. {'Less.'} 100
Boiss. {'Boiss.'} 100
(Willd.) {'(Willd.)', 'Boiss.'} 90
Boiss. {'(Willd.)', 'Boiss.'} 90
Mout. {'Mouterde'} 90
Mobayen {'Mobayen'} 100
Boiss. {'Boiss.'} 100
(Schultes) {'(Willd.', 'Boiss.', 'Schult.)'} 77
Boiss. {'(Willd.', 'Boiss.', 'Schult.)'} 90
L. {'L.'} 100
L. {'L.'} 100
L. {'L.'} 100
Boiss. {'Boiss.'} 100
L. {'L.'} 100
DC. {'DC.'} 100
np.NaN set() 0
Afan. {'Afanasiev'} 90
DC. {'DC.'} 100
L. {'L.'} 100
Schultz {'Sch.Bip.', '(Forssk.)'} 51
Bip. {'Sch.Bip.', '(Forssk.)'} 90
(Forssk.) {'Sch.Bip.', '(Forssk.)'} 90
Boiss. {'Boiss.'} 100
(Labili.). {'DC.'} 0
DC. {'DC.'} 100
DC. {'DC.'} 100
Koch {'K.Koch'} 90
C. {'K.Koch'} 90
Cass. {'Cass.'} 100
Cass. {'Cass.', '(L.)'} 90
(L.) {'Cass.', '(L.)'} 60
L. {'L.'} 100
Davis {'P.H.Davis'} 90
Schreb. {'Schreb.'} 100
Boiss. {'Boiss.'} 100
L. {'L.'} 100
Bilik {'Bilik'} 100
Benth. {'Benth.', 'Ging.'} 90
Gingins {'Benth.', 'Ging.'} 66
Tausch {'Tausch'} 100
Sam. {'Sam.', 'Rech.f.'} 90
Rech. {'Sam.', 'Rech.f.'}

KeyboardInterrupt: 

In [87]:
vol1_df['closest_wcvp_match'] = vol1_df.apply(get_closest_match, axis = 1)
print("vol1 complete")
vol2_df['closest_wcvp_match'] = vol2_df.apply(get_closest_match, axis = 1)
print("vol2 complete")
vol3_df['closest_wcvp_match'] = vol3_df.apply(get_closest_match, axis = 1)
print("vol3 complete")

vol1 complete
vol2 complete
vol3 complete


In [422]:
"hi"

'hi'